Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from scipy.stats import uniform
from sklearn.decomposition import TruncatedSVD


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [4]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [5]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
2174,4030,"\nThis blend has been a Whyte and MacKay whisky for 25 years. The nose has spice, toffee, wheat biscuits, and smoke, but the combination feels a little benign. The palate is jammy, lightweight, and middle-of-the-road. With a backbone of grain, there are flavors of graham cracker, chocolate ganache, well-baked fruitcake, banoffee pie, smoke, and dried peels, with a burst of spices on the finish.",0
174,4247,"\nLocated in the heart of Vermont’s syrup belt, Saxtons River distillery makes both a Vermont maple-flavored rye and a maple-flavored bourbon whiskey. The rye is the better of the two and does a solid job of mixing oak, cinnamon, black pepper, and rye spice notes with vanilla and rich maple syrup. The rye whiskey is a little young, but the maple syrup complements it well and is more companion than cover.",0
3463,4909,"\nExplore the grain side of Jameson’s personality in this first specimen from the Deconstructed series. Expect floral aromatics and a mixed crate of fresh citrus fruit, so it smells closer to the airport’s perfume counter than the whiskey store. Strands of honey, squeezed citrus, and vanilla blossom amid a thickening mouthfeel after a feather-light beginning. It’s tooth-achingly sweet, but daubed with lime and sweet clementine, leading to a dry, warm finish with chalky candy. (Travel Retail exclusive) €36",0


In [6]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
1252,5053,"\nSo that’s what wheat smells like: not much. The aromas here seem to be mostly yeast-derived, with a faint ripe fruitiness (undefined: peach, apple?) and alcohol heat, and a touch of wheat-origin sweet grass. Wheat’s in the mouth, like chewing fresh grain with some water and alcohol. It’s all there, and the finish is sweet, and it’s clean, but…that’s about all. White bread whiskey.",2
3867,5091,"\nEarthy, fleshy notes with brooding grape notes and a gamey finish -- the Nebbiolo grape influence is obvious. I enjoy Barolo wine and I enjoy Edradour whisky, but the flavors in this whisky mix like oil and water. (Exclusive to Binny’s Beverage Depot.)",2
2807,5087,"\nAged in a sherry cask, which adds sweet notes of toffee, Caribbean rum, waxed fruit, red licorice, and fruitcake to go along with the earthy smoke and lingering peat. A much sweeter, fruitier affair than the Croftengea also reviewed here, but behind all that sherry there’s still a whisky that needs a little more maturing. \r\n",2


### Split the Training Set into Train/Validation

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3269,) (3269,) (818,) (818,)


In [10]:
from sklearn.pipeline import Pipeline

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2)) #vectorizer
clf = LinearSVC() #classifier

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [11]:
parameters = {
    'vect__max_df': (0.3, .5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
    'clf__penalty': ('l1','l2'),
    'clf__C': (0.1, 0.5, 1., 2.)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [12]:
grid_search.best_score_

0.7575254449309149

In [13]:
grid_search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vect', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__norm', 'estimator__vect__preprocessor', 'estimator__vect__smooth_idf', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__sublinear_tf', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__use_idf', 'estimator__vect__vocabulary', 'estimator__clf__C', 'estimator__clf__class_weight', 'estimator__clf__dual', 'estimator__clf__fit_intercept', 'estimator__clf__intercept_scaling', 'estimator__clf__loss', 'estimator__clf__max_iter', 'estimator__clf__multi_class', 'estimato

In [14]:
params = grid_search.best_params_

In [15]:
params

{'clf__C': 0.5,
 'clf__penalty': 'l2',
 'vect__max_df': 0.5,
 'vect__max_features': 20000,
 'vect__min_df': 2}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [16]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [17]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [18]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [19]:
subNumber = 0

In [20]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


In [21]:
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,random_state=0)
distributions = dict(C=uniform(loc=0, scale=4),penalty=['l2', 'l1'])
RSCV = RandomizedSearchCV(logistic, distributions, random_state=0)

In [22]:
RSCV.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__class_weight', 'estimator__dual', 'estimator__fit_intercept', 'estimator__intercept_scaling', 'estimator__l1_ratio', 'estimator__max_iter', 'estimator__multi_class', 'estimator__n_jobs', 'estimator__penalty', 'estimator__random_state', 'estimator__solver', 'estimator__tol', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'iid', 'n_iter', 'n_jobs', 'param_distributions', 'pre_dispatch', 'random_state', 'refit', 'return_train_score', 'scoring', 'verbose'])

### Define Pipeline Components

In [34]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
svd = TruncatedSVD(algorithm='randomized', n_iter=10) 
rfc = RandomForestClassifier(n_jobs = -1)

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', rfc)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [51]:
g_params = {
    'clf__max_depth': (10,42,None),
    'clf__n_estimators': (11,21,31,41,51),
    'clf__max_leaf_nodes':(10,20,30,40,60,80,None),
    'vect__max_df': (0.3, .5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
}

In [57]:
GSCV_search = GridSearchCV(pipe, g_params, cv=5, n_jobs=-1, verbose=1)
GSCV_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 1260 candidates, totalling 6300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 20.3min
[Parallel(n_jobs=-1)]: Done 6300 out of 6300 | elapsed: 21.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [58]:
GSCV_search.best_score_

0.7151930403138682

In [52]:
rando_search = RandomizedSearchCV(pipe, r_params, cv=5, n_jobs=-1, verbose=1, random_state = 17)
rando_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   10.9s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [53]:
rando_search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vect', 'estimator__svd', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__norm', 'estimator__vect__preprocessor', 'estimator__vect__smooth_idf', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__sublinear_tf', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__use_idf', 'estimator__vect__vocabulary', 'estimator__svd__algorithm', 'estimator__svd__n_components', 'estimator__svd__n_iter', 'estimator__svd__random_state', 'estimator__svd__tol', 'estimator__clf__bootstrap', 'estimator__clf__ccp_alpha', 'estimator__clf__clas

In [54]:
rando_search.best_params_

{'vect__min_df': 2,
 'vect__max_features': 20000,
 'vect__max_df': 0.5,
 'clf__n_estimators': 21,
 'clf__max_leaf_nodes': 40,
 'clf__max_depth': None}

In [55]:
rando_search.best_score_

0.7066295978189632

### Make a Submission File

In [59]:
# Predictions on test sample
pred = rando_search.predict(test['description'])

In [59]:
# Predictions on test sample
pred = rando_search.predict(test['description'])

In [60]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [61]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [63]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

In [64]:
# Predictions on test sample
pred = GSCV_search.predict(test['description'])

In [65]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [66]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [67]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt7/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?